# 1. Import packages

In [1]:
import glob
import json
import os
import pandas as pd

# 2. Read a txt file

## 2.1. Change to inputs directory

In [2]:
print(os.getcwd())
input_path_s = "/inputs/logs"
input_path_s = os.getcwd() + input_path_s
output_path_s = "/outputs"
output_path_s = os.getcwd() + output_path_s
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task14_de_BrightData
/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task14_de_BrightData/inputs/logs


## 2.2. Read all txt log files and collect them in a dictionary

In [3]:
log_d = dict()

for file in glob.glob("*.txt"):
    
    # Get the txt filename without the extension
    txt_filename = os.path.splitext(file)[0]
    
    if txt_filename[:1].isdigit():
    
        # Parse AS number from txt_filename
        dot_loc_i = txt_filename.find(".")
        as_number_s = txt_filename[:dot_loc_i]

        # If there is a number at the end of the filename, then remove.
        if txt_filename[-1].isdigit():
            txt_filename = txt_filename[:-1]

        # Try whether txt_filename is in the dictionary
        # If not, then assign into the dictionary
        try:
            log_d[txt_filename]
            print(f"DUPLICATE LOG FILE: {txt_filename}")
        except:
            if os.path.getsize(file) > 500:
                log_d[txt_filename] = pd.read_csv(file, sep="\n", header=None, on_bad_lines='skip')
            else:
                log_d[txt_filename] = pd.DataFrame()
    
print(len(log_d))

DUPLICATE LOG FILE: 2022071
8009


# 3. Count Request URL: FQDN

In [4]:
def produce_fqdn(request_series):
    '''
    parameter: path of txt files
    output: list of txt files
    does: Read all txt files in the path and creates a list of them
    '''
    fqdn_set = set()
    is_reachable = True
    for req in request_series:
        if req[:15] == "{'request url':":
            slash_loc_i = req.find("/")
            comma_loc_i = req.find(",")
            url = req[slash_loc_i+2:comma_loc_i-1]

            slash_loc_i = url.find("/")
            if slash_loc_i > 0:
                url = url[:slash_loc_i]

            question_mark_loc_i = url.find("?")
            if question_mark_loc_i > 0:
                url = url[:question_mark_loc_i]

            if "." in url:
                fqdn_set.add(url)
        else:
            if req[-9:] == "<unknown>":
                is_reachable = False         
            
    return fqdn_set, is_reachable

In [5]:
country_l = []
domain_l = []
as_number_l = []
n_fqdn_response_l = []
is_reachable_l = []

for log_name, resp_df in log_d.items():
    # Parse country, target domain, and AS number based on log_name
    dot_loc_i = log_name.find(".")
    as_number_i = int(log_name[:dot_loc_i])
    country_s = log_name[dot_loc_i+1:dot_loc_i+3]
    domain_s = log_name[dot_loc_i+4:]
    
    if resp_df.empty:
        fqdn_set = set()
        is_reachable = False
    else:
        # Convert the DataFrame into Series
        # Please note that the column name in DataFrame is 0.
        resp_ser = resp_df[0].squeeze()
        fqdn_set, is_reachable = produce_fqdn(resp_ser)

    # Collect into lists
    country_l.append(country_s)
    domain_l.append(domain_s)
    as_number_l.append(as_number_i)
    n_fqdn_response_l.append(len(fqdn_set))
    is_reachable_l.append(is_reachable)

# 4. Create a CSV file

In [6]:
col_l = ["country", "target_domain", "as_number", "#fqdn_in_response", "is_reachable"]

In [7]:
df = pd.DataFrame(list(zip(country_l, domain_l, as_number_l, n_fqdn_response_l, is_reachable_l)), columns = col_l)

In [8]:
print(os.getcwd())
os.chdir(output_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task14_de_BrightData/inputs/logs
/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task14_de_BrightData/outputs


In [9]:
df.to_csv("de.csv", index=False)